## Aplicación de charapy en caso práctico

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('com.csv')

In [3]:
carbon_number = df['component'].to_numpy()
molar_fraction = df['molar_percentage'].to_numpy()
molecular_weight = df['molar_mass'].to_numpy()
density = df['density'].to_numpy()

carbonnumber_max = carbon_number[-2]
molar_fraction_max = molar_fraction[-2]
molecular_weight_max = molecular_weight[-2]
density_max = density[-2]
molar_fraction_plus = molar_fraction[-1]
molecular_weight_plus = molecular_weight[-1]

Primero ajusto para obtener todas las constantes intervinientes. Para eso utilizo la librería Foo_fit.

Dato: Si no quisiera obtener o chequear los valores de los parámetros de ajuste puedo pasar directamente a la distribución y al no darle a la función un valor para las constantes las calcula automáticamente a partir de un diccionario definido en la clase.

En este ejemplo, por cuestiones de evaluación se calculan y se evidencian

In [4]:
from charapy import Foo_fit
foo_fit = Foo_fit()

A,B = foo_fit.fit_AB(carbon_number,molar_fraction)
L,M = foo_fit.fit_LM(carbon_number, density)
Ac, Bc = foo_fit.fit_AcBc(carbon_number, molar_fraction)
Ad = foo_fit.fit_Ad(carbon_number, density)[0]
C = foo_fit.fit_C(carbon_number,molecular_weight) [0]

fit_parameters = {
    'A':A, 'B':B, 'L':L, 'M':M, 'Ac':Ac, 'Bc':Bc
    ,'Ad':Ad,'C':C
}

fit_parameters

{'A': 20.24141696010174,
 'B': -5.350895658855276,
 'L': 0.5181480391330261,
 'M': 0.11698833413234153,
 'Ac': 2.9969050222015183,
 'Bc': -87.60869574376241,
 'Ad': -0.4578051749233114,
 'C': 13.913346938986416}

Para no utilizar los valores establecidos aleatoriamente por Pedersen para el número máximo de carbono
calculo el valor máximo con el criterio de Cismondi et. al. Este método está definido en la librería Residual_fraction.

In [5]:
from charapy import Residual_fraction

residual_fraction = Residual_fraction(molecular_weight_plus,molar_fraction_plus)

carbon_range_it = np.array(range(carbonnumber_max,10000,1))
res_carbon_number_mx = residual_fraction.carbon_number_max(
    carbon_range_it, A, B, C
    )

print('Número máximo de carbono calculado:', res_carbon_number_mx)

Número máximo de carbono calculado: 40


Aplico las funciones de distribución para obtener todas las propiedades extendidas del fluido que se requieren para ingresar luego a la EoS. 

Se realiza para el criterio de Pedersen y para las modificaciones establecidas por Cismondi, cada uno con su correspondiente librería: Distribution_pedersen y Distribution_cismondi.

In [6]:
from charapy import Distribution_pedersen, Distribution_cismondi

distribution_pedersen = Distribution_pedersen()
distribution_cismondi = Distribution_cismondi()

c_range = np.arange(carbonnumber_max+1, res_carbon_number_mx+1)
ped_density = distribution_pedersen.p_density(c_range,L,M)
cis_density = distribution_cismondi.c_density(c_range, Ad)

ped_mw = distribution_pedersen.p_molecular_weight(c_range)
cis_mw = distribution_cismondi.c_molecular_weight(c_range,C)

ped_mf = distribution_pedersen.p_molar_fraction(c_range,A,B)
cis_mf = distribution_cismondi.c_molar_fraction(c_range, Ac, Bc)

Con este procedimiento ya resolvimos la caracterización de los datos distribuidos. ¿Qué logramos? Distribuir la fracción plus de la muestra de petroleo (aquella fracción que no podemos caracterizar en el laboratorio por la dificultad para separar sus partes), expandir los datos y estamos listos para obtener las propiedades que vamos a utilizar para ingresar a una Ecuación de Estado. 

La distribución para los datos analizados sólo nos permitió expandir óptimamente 3 números de carbono, esto tiene que ver con el criterio de corte en donde establezco que se detenga la distribución cuando se cumple el balance de masa. Esto debe ajustarse con una metodología planteada en el paper de cismondi et. al. y la flexibilidad del paquete me permite realizarlo (es una tarea en la que estoy trabajando que luego modificará la función carbon_number_max)

In [7]:
df ['molar_percentage_pedersen'] = df ['molar_percentage']
df ['molar_percentage_cismondi'] = df ['molar_percentage']
df ['molecular_weight_pedersen'] = df ['molar_mass']
df ['molecular_weight_cismondi'] = df ['molar_mass']
df ['density_pedersen'] = df ['density']
df ['density_cismondi'] = df ['density']

df = df.drop(['molar_mass','molar_percentage','density'],  axis=1)

In [8]:
df['Tc_pedersen'] = np.nan
df['Tc_cismondi'] = np.nan
df['Pc_pedersen'] = np.nan
df['Pc_cismondi'] = np.nan

### Correlaciones para determinar Tc, Pc, factor acéntrico etc. 

In [9]:
from charapy import Correlations

correlations = Correlations('PR')

tc_ped = correlations.critical_temperature(ped_mw,ped_density)
tc_cis = correlations.critical_temperature(cis_mw,cis_density)
pc_ped = correlations.critical_pression(ped_mw, ped_density)
pc_cis = correlations.critical_pression(cis_mw,cis_density)

In [10]:
for i in range (len(c_range)):
    df=pd.concat([df,
        pd.DataFrame(
        {'component': c_range[i],
        'molar_percentage_pedersen': ped_mf[i],
        'molar_percentage_cismondi' : cis_mf[i],
        'molecular_weight_pedersen': ped_mw[i],
        'molecular_weight_cismondi': cis_mw[i],
        'density_pedersen': ped_density[i],
        'density_cismondi': cis_density[i], 
        'Tc_pedersen': tc_ped[i],
        'Tc_cismondi': tc_cis[i],
        'Pc_pedersen': pc_ped[i],
        'Pc_cismondi': pc_cis[i]   
        }, index = [c_range[i]])])
df

,component,molar_percentage_pedersen,molar_percentage_cismondi,molecular_weight_pedersen,molecular_weight_cismondi,density_pedersen,density_cismondi,Tc_pedersen,Tc_cismondi,Pc_pedersen,Pc_cismondi
0,7,2.870000,2.870000e+00,96.0,96.000000,0.738000,0.738000,NaN,NaN,NaN,NaN
1,8,4.080000,4.080000e+00,107.0,107.000000,0.765000,0.765000,NaN,NaN,NaN,NaN
2,9,3.510000,3.510000e+00,121.0,121.000000,0.781000,0.781000,NaN,NaN,NaN,NaN
3,10,3.260000,3.260000e+00,134.0,134.000000,0.792000,0.792000,NaN,NaN,NaN,NaN
4,11,2.510000,2.510000e+00,147.0,147.000000,0.796000,0.796000,NaN,NaN,NaN,NaN
5,12,2.240000,2.240000e+00,161.0,161.000000,0.810000,0.810000,NaN,NaN,NaN,NaN
6,13,2.180000,2.180000e+00,175.0,175.000000,0.825000,0.825000,NaN,NaN,NaN,NaN
7,14,2.070000,2.070000e+00,190.0,190.000000,0.836000,0.836000,NaN,NaN,NaN,NaN
8,15,2.030000,2.030000e+00,206.0,206.000000,0.842000,0.842000,NaN,NaN,NaN,NaN
9,16,1.670000,1.670000e+00,222.0,222.000000,0.849000,0.849000,NaN,NaN,NaN,NaN


In [11]:
from charapy import Lumping

lumping = Lumping()
dat = lumping.lumpy([(7,11),(12,14),(15,18),(19,22),(23,25),(26,30),(31,34),(35,40)],df,'component')
dat

,component,molar_percentage_pedersen,molar_percentage_cismondi,molecular_weight_pedersen,molecular_weight_cismondi,density_pedersen,density_cismondi,Tc_pedersen,Tc_cismondi,Pc_pedersen,Pc_cismondi
ID,,,,,,,,,,,
0.0,"(7, 11)",3.246000,3.246000e+00,121.000000,121.000000,0.774400,0.774400,NaN,NaN,NaN,NaN
1.0,"(12, 14)",2.163333,2.163333e+00,175.333333,175.333333,0.823667,0.823667,NaN,NaN,NaN,NaN
2.0,"(15, 18)",1.610000,1.610000e+00,229.000000,229.000000,0.846000,0.846000,NaN,NaN,NaN,NaN
3.0,"(19, 22)",0.970000,9.700000e-01,283.500000,283.500000,0.865500,0.865500,NaN,NaN,NaN,NaN
4.0,"(23, 25)",0.640000,6.400000e-01,331.333333,331.333333,0.881000,0.881000,NaN,NaN,NaN,NaN
5.0,"(26, 30)",2.006904,3.639797e+00,398.016667,398.336721,0.914008,0.913576,906.788383,908.257384,1.989448,1.953352
6.0,"(31, 34)",0.103393,4.212602e+05,451.000000,452.703694,0.925343,0.918387,937.310965,938.240009,2.082022,1.966700
7.0,"(35, 40)",0.041795,1.810080e+13,521.000000,522.270429,0.942032,0.925325,996.460924,996.269230,2.389751,1.988068
